# Linear Regression

Import all the packages here.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## CPU Peformance Data Set

In [ ]:
# Let's download some data to play with
# source: UCI ML Repo
!mkdir ./data/cpu_perf
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data > ./data/cpu_perf/machine.data
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.names > ./data/cpu_perf/machine.names

In [ ]:
!cat data/cpu_perf/machine.names

## Review Exercise

In [ ]:
columns = ['vendor', 'model', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'PRP', 'ERP']

In [ ]:
# Read the data
path = './data/cpu_perf/machine.data'

# this specifies data types for each column to avoid mis-interpretation of data type by Pandas
dtype = {
    **{i: 'category' for i in columns[:1]},
    **{i: int for i in columns[2:]}
}

df = pd.read_csv(path, names=columns, dtype=dtype)

In [ ]:
# Preview the first five rows of the DataFrame.
df.head()

In [ ]:
# check column data type


In [ ]:
# what is the size of the dataset? a.k.a. how many rows and columns are in the data


In [ ]:
# how many vendors are in the dataset?


In [ ]:
# how many models are in the dataset?


In [ ]:
# other relationships you want to look into


<a id="visualizing-the-data"></a>
## Our First Model

In [ ]:
# make a scatterplot of `CHMAX` against `ERP`
_ = plt.subplots(figsize=(12, 8))
_ = plt.scatter(df.CHMAX, df.ERP)
_ = plt.xlabel('Maximum Channels in Unit')
_ = plt.ylabel('Relative CPU Performance')

In [ ]:
# Use Seaborn to create a scatterplot with regression line
# sns.lmplot does not allow us to pass in an Axes object.
# It creates a FaceGrid object that has the relevant Axes object
# as its ax attribute.
sns.set_style("whitegrid")
sns.lmplot(x='CHMAX', y='ERP', data=df)

We just created a linear regression model!

- **Formula for a line:** $y = mx + b$
- **Alternative notation:** $y = \beta_0 + \beta_1 * x$
- **Our model:** <br>
    $\mbox{CPU_Performance} = \beta_0 + \beta_1 * \mbox{Maximum_Number_of_Channels} + \epsilon$

We call $\beta_0$ the **model intercept** and $\beta_1$ the **coefficient of `CHMAX`**.

We call $\epsilon$ the **error term**. It accounts for the fact that our points do not lie exactly on a line. Linear regression is designed to be optimal when this noise is normally distributed with constant variance. We ignore it when we use the model to make predictions.

**Q&A** What would our model predict for `ERP` when `CHMAX` is 100?

---

**Takeaways:**

- Linear regression with one input feature chooses the *line* that "best fits" a scatterplot of the target variable against that input feature.
- The model intercept tells you what the model would predict if all of the input variables were zero.
- The coefficient on a variable tells you how much and in what direction the model's prediction would change if the relevant variable were to increase by 1.

# Building a Linear Regression Models with scikit-learn

When we called `sns.lmplot`, `seaborn` created a linear regression model which it then displayed to us.

A more typical workflow for creating a linear regression model uses **scikit-learn**.

[scikit-learn](https://scikit-learn.org/) is the most popular Python library for machine learning.

**Strengths:**

- Includes good implementations of a wide range of algorithms.
- Provides consistent interface across model types.
- Provides excellent documentation.
- Large community --> tons of resources for learning and getting questions answered.

**Limitations:**

- Designed primarily for single-thread, in-memory computing.
- Has only basic capabilites for deep learning.
- Reflects machine learning rather than statistics mindset: focuses on predictive accuracy on held-out data rather than hypothesis testing, parameter estimation, and model interpretation.

In [ ]:
# 1. Import the LinearRegression model class.
from sklearn.linear_model import LinearRegression

In [ ]:
# 2. Make an instance of the LinearRegression class.
lrg = LinearRegression()

In [ ]:
# 3. Train the model instance on our data to recreate what we just did in the previous step
lrg.fit(df[['CHMAX']], df.ERP)

In [ ]:
# 4. Use the model to make predictions.
y_pred = lrg.predict(df[['CHMAX']])
y_pred[:5]

In [ ]:
# Let's compare fitted values to actual.
result = df.ERP.to_frame().assign(predicted=y_pred, error=y_pred-df.ERP)
result.head(10)

In [ ]:
# Another way to see how accurate our model performs
_ = plt.subplots(figsize=(12, 8))
_ = plt.scatter(result.ERP, result.predicted)
_ = plt.xlabel('Actual Value')
_ = plt.ylabel('Predicted Value')

**Q&A** What would the plot above look like if our model's predictions were **perfectly accurate**?

---

<a id="scikit-learns--step-modeling-pattern"></a>
### scikit-learn's Four-Step Modeling Pattern

1. Import the class you plan to use.
2. "Instantiate" the class. You can specify "hyperparameters" at this point.
3. Fit the model instance with data. (This step changes the model object in-place.)
4. Use the fitted model to make predictions.

# Linear Regression with Multiple Features

- **Using `CHMAX` only:** <br>$\mbox{ERP} = \beta_0 +  \beta_1 * \mbox{CHMAX} + \epsilon$
- **Using `CHMAX` and `CHMIN`:** <br>$\mbox{ERP} = \beta_0 +  \beta_1 * \mbox{CHMIN} +  \beta_2 * \mbox{CHMAX} + \epsilon$

Linear regression with two input features chooses the "plane" that "best fits" a 3D scatterplot of the target variable against those input features.

In general, linear regression chooses the "hyperplane" that "best fits" a scatterplot of the target variable against the input features.

**Exercise.** Answer each question below for the second model written out above, in terms of $\beta_0$, $\beta_1$, $\beta_2$, and $\epsilon$ ("epsilon").

- What would our model predict for `ERP` at `CHMIN=0` and `CHMAX=0`?

- If `CHMIN` increases by 5 while `CHMAX` remains the same, how does our model's prediction for `ERP` change?

- If `CHMAX` decreases by 3 while `CHMIN` remains the same, how does our model's prediction for `ERP` change?

- **Bonus:** Why does the previous question say "while `CHMIN` remains the same?"

More on [Ceteris Paribus](https://www.investopedia.com/terms/c/ceterisparibus.asp).

**Takeaways:** Interpretation of the parameters of a linear regression model does not change as we add more variables, except that each coefficient tells us how the model's prediction would change if the associated variable were to change *while all other variables remained the same*.

---

### Exercise

Build another linear regression model, this time using all of our features as input instead of just `temp_celsius`, except `vendor` and `model`.

In [ ]:
df.head()

**Model using all columns as given:** 

$\mbox{ERP} = \beta_0 + \beta_1 * \mbox{MYCT} + \beta_2 * \mbox{MMIN} + \beta_3 * \mbox{MMAX} + \beta_4 * \mbox{CACH} + \beta_5 * \mbox{CHMIN} + \beta_6 * \mbox{CHMAX} + \beta_7 * \mbox{PRP} + \epsilon$

- Store a pandas DataFrame with the values of the feature variables (everything except `vendor`, `model` and `ERP`) as a Python variable X.

In [ ]:
columns

In [ ]:
X = df[columns[2:-1]]
y = df[columns[-1]]

In [ ]:
X.head()

In [ ]:
y.head()

- Make a new instance of the LinearRegression class. Call it lr_all to distinguish it from our last model.

- Train the model instance using our new feature matrix $X$ and the same target variable $y$.

- Store `lr_all`'s fitted values in a new `predictions` column of the `df` DataFrame.

In [ ]:
df['predicted'] = 

In [ ]:
# Compare predicted values to actual
_ = plt.subplots(figsize=(12, 8))
_ = plt.scatter(df.ERP, df.predicted)
_ = plt.xlabel('Actual Value')
_ = plt.ylabel('Predicted Value')

The plot above shows how well the model fits the data it was trained on. How well it would predict new data that it wasn't trained on is a further issue.

In [ ]:
# Drop the fitted values from our DataFrame
df.drop('predicted', 1, inplace=True)

---

#### Explore the intercept and coefficients of the linear model

In [ ]:
# Print intercept
lr_all.intercept_

In [ ]:
# Print coefficient
pd.DataFrame(zip(columns[2:-1], lr_all.coef_), columns=['variable', 'coefficient'])

In [ ]:
# Look at the documentation for a LinearRegression model object
help(LinearRegression)

# A Little Theory

Fitting a linear regression selects the coefficients and intercept that minimize the **sum of squared errors** of the fitted values on the training set.

![Estimating coefficients](./img/estimating_coefficients.png)

In the diagram above:

- The black dots are the **observed values** of x and y.
- The blue line is our **least squares line**.
- The red lines are the **residuals**, which are the vertical distances between the observed values and the least squares line.

**Justification:** Minimizing the sum of squared errors maximizes *the probability of the data given the model* (call the **likelihood** of the model on the data) on the assumption that the target variable really is a linear function of the features plus normally distributed noise: $y = \beta_0 + \sum \beta_i x_i + \epsilon$ where $\epsilon \sim \mathcal{N}(0, \sigma^2)$

![](./assets/400px-Linear_regression.svg.png)

<a id="evaluation-metrics-for-regression-problems"></a>
### Evaluation Metrics for Regression Problems

**Mean absolute error (MAE)** is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

**Mean squared error (MSE)** is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

**Root mean squared error (RMSE)** is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

**Exercise.** Calculate MAE, MSE, and RMSE for the values below.

In [ ]:
# Example true and predicted response values
true = y.copy().values
pred = lr_all.predict(X)

In [ ]:
# calculating errors by hand
mae = np.average(np.abs(true - pred))
mse = np.average(np.power(true-pred, 2))
rmse = np.sqrt(mse)

In [ ]:
# compare manual work with pre-implemented work
from sklearn import metrics

print(mae)
np.testing.assert_almost_equal(mae, metrics.mean_absolute_error(true, pred))

print(mse)
np.testing.assert_almost_equal(mse, metrics.mean_squared_error(true, pred))

print(rmse)
np.testing.assert_almost_equal(rmse, np.sqrt(metrics.mean_squared_error(true, pred)))

Let's compare these metrics:

- MAE is the easiest to understand, because it's the average error.
- MSE is more popular than MAE, primary because the fact that it is continuous and differentiable makes it easier to work with.
- RMSE is even more popular than MSE, because RMSE is interpretable in the "y" units.
- MSE/MSE punishes large errors more than MAE.

**R-squared** is a metric that directly addresses the question of how well your model does compared to the null model, which usually is just the average of the target variable.

$$R^2=1-\frac{\mbox{Mean Squared Error}}{\mbox{Mean Squared Total}}$$

"Mean Squared Total" is exactly the MSE of the null model, which is essentially the variance of the data.

**R-squared properties:**

- *Typically* greater than 0 (unless your model is worse than the null model, which can happen) or read more about it in this [article](http://www.fairlynerdy.com/what-is-r-squared/)
- Never greater than 1

![](./assets/r_squared.png)

**Aside:** Despite common usage, R-squared *does not tell you how much of the variance in the data the model explains*, at least in the normal sense of "explains."

Explanations are usually *causal*, and R-squared *does nothing to distinguish between causation and association*.

In [ ]:
# Use the sklearn .score() method to calculate test-set R-squared
?lr_all.score

In [ ]:
lr_all.score(X, y)

To interpret,<br>
    1) Our model is 95.95% better than the null model (**preferred**)<br>
    2) Our model explains 95.95% of variance in the dataset<br>

## What to learn next: Regularization

Note: This is an advanced concept that is out of scope of the workshop. It is strongly encouraged to learn more about this after you are comfortable with linear regression concepts.

- Overly complicated models don't generalize well.
- One way to reduce model complexity is to use fewer features.
- Another way is to incorporate a penalty for coefficient size.

<a id="how-does-regularization-work"></a>
### How Does Regularization Work?

For a normal linear regression model, we estimate the coefficients using the least squares criterion, which minimizes the residual sum of squares (RSS).

For a regularized linear regression model, we minimize the sum of MSE and a "penalty term" that penalizes coefficient size.

**Ridge regression** (or "L2 regularization") minimizes: $$\mbox{MSE} + \alpha \sum_{j=1}^p \beta_j^2$$

**Lasso regression** (or "L1 regularization") minimizes: $$\text{Mse} + \alpha \sum_{j=1}^p |\beta_j|$$

- $p$ is the number of features.
- $\beta_j$ is a model coefficient.
- $\alpha$ is a tuning parameter:
    - A tiny $\alpha$ imposes no penalty on the coefficient size, and is equivalent to a normal linear regression model.
    - Increasing the $\alpha$ penalizes the coefficients and thus shrinks them.

To read more about Ridge and Lasso Regression, check out this awesome [article](https://www.datacamp.com/community/tutorials/tutorial-ridge-lasso-elastic-net) by DataCamp.